In [1]:
import sys

sys.path.append('../../code/')
import os
import json
from datetime import datetime
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

import networkx as nx

from load_data import load_citation_network, case_info
from helper_functions import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

data_dir = '../../data/'
court_name = 'scotus'

In [22]:
court_adj_mat = pd.read_csv(data_dir + 'clean/jurisdictions_adj_mat.csv', index_col='Unnamed: 0')
court_adj_mat.index = [j + '_ing' for j in court_adj_mat.index]
court_adj_mat.columns= [j + '_ed' for j in court_adj_mat.columns]

# Grab federal appellate subnetwork

In [44]:
fed_appellate = ['ca' + str(i+1) for i in range(11)]
fed_appellate.append('cafc')
fed_appellate.append('cadc')

fed_appellate_ing = [j + '_ing' for j in fed_appellate]
fed_appellate_ed = [j + '_ed' for j in fed_appellate]

In [45]:
fed_appellate_network = court_adj_mat.loc[fed_appellate_ing, fed_appellate_ed]

In [46]:
fed_appellate_network

,ca1_ed,ca2_ed,ca3_ed,ca4_ed,ca5_ed,ca6_ed,ca7_ed,ca8_ed,ca9_ed,ca10_ed,ca11_ed,cafc_ed,cadc_ed
ca1_ing,151702,6656,8507,5928,14008,6875,11991,8486,13901,6697,5072,570,5575
ca2_ing,4229,93422,4849,3134,5570,3374,5918,3715,7092,2879,2226,272,3307
ca3_ing,9683,9711,183320,7779,17799,8669,14532,9541,16940,7783,6017,895,6501
ca4_ing,6677,5920,7749,186566,13910,7350,11521,8450,12537,6601,5771,471,5605
ca5_ing,8541,9195,19136,8885,434933,9263,13788,11229,17853,8531,6696,665,7818
ca6_ing,8227,7556,9942,7980,16663,262366,15882,11250,16922,7979,6346,671,5900
ca7_ing,9839,10101,12246,8392,17765,10952,326501,13214,19171,9299,7310,773,7766
ca8_ing,6857,6463,7917,6813,14228,7754,12830,282702,14574,7916,4992,497,5438
ca9_ing,9664,10656,12182,8566,20039,9710,15850,11809,468980,9775,6702,1179,10503
ca10_ing,7021,5248,7746,5983,13616,7149,11674,9973,14871,237590,5536,600,5139


# get inter court favorability rankings

In [73]:
inter_juris_rankings = pd.DataFrame(columns = fed_appellate)

for court in fed_appellate:

    # grab the inter court citations for court
    citing_count = fed_appellate_network.loc[court + '_ing']
    
    # rank the courts by number of citations
    court_ranking = citing_count.sort_values(inplace=False, ascending=False).index.tolist()
    inter_juris_rankings[court] = [j.split('_')[0] for j in court_ranking]
    
inter_juris_rankings.index = [i + 1 for i in inter_juris_rankings.index]
inter_juris_rankings.index.name = 'rank'

In [74]:
inter_juris_rankings

,ca1,ca2,ca3,ca4,ca5,ca6,ca7,ca8,ca9,ca10,ca11,cafc,cadc
rank,,,,,,,,,,,,,
1,ca1,ca2,ca3,ca4,ca5,ca6,ca7,ca8,ca9,ca10,ca11,cafc,cadc
2,ca5,ca9,ca5,ca5,ca3,ca9,ca9,ca9,ca5,ca9,ca5,ca9,ca5
3,ca9,ca7,ca9,ca9,ca9,ca5,ca5,ca5,ca7,ca5,ca9,ca5,ca9
4,ca7,ca5,ca7,ca7,ca7,ca7,ca8,ca7,ca3,ca7,ca7,cadc,ca7
5,ca3,ca3,ca2,ca8,ca8,ca8,ca3,ca3,ca8,ca8,ca3,ca3,ca3
6,ca8,ca1,ca1,ca3,ca6,ca3,ca6,ca10,ca2,ca3,ca8,ca7,ca8
7,ca6,ca8,ca8,ca6,ca2,ca1,ca2,ca6,cadc,ca6,ca1,ca1,ca2
8,ca10,ca6,ca6,ca1,ca4,ca4,ca1,ca1,ca10,ca1,ca6,ca2,ca4
9,ca2,cadc,ca10,ca10,ca1,ca10,ca10,ca4,ca6,ca4,ca4,ca8,ca1


# load jurisdiction network into igraph

In [75]:
import igraph as ig

In [80]:
court_adj_mat.as_matrix() + court_adj_mat.transpose().as_matrix()

array([[500928,  95606,  84758, ...,     56,      1,      0],
       [ 95606, 303404,  10885, ...,     25,      0,      0],
       [ 84758,  10885, 186844, ...,     28,      0,      0],
       ..., 
       [    56,     25,     28, ...,   3062,      0,      0],
       [     1,      0,      0, ...,      0,      0,      0],
       [     0,      0,      0, ...,      0,      0,      0]])

In [92]:
num_jurisdictions = court_adj_mat.shape[0]

courts = [j.split('_')[0] for j in court_adj_mat.index.tolist()]

In [131]:

G = ig.Graph(n=num_jurisdictions)
G.vs['name'] = courts

# list of all edges
edges = get_pairs(jurisdictions)
for e in edges:
    ct1 = e[0]
    ct2 = e[1]

    weight = court_adj_mat.loc[ct1 + '_ing', ct2 + '_ed'] + court_adj_mat.loc[ct2 + '_ing', ct1 + '_ed']
    
    if weight > 0:
        v1 = G.vs.select(name=ct1)[0]
        v2 = G.vs.select(name=ct2)[0]
        
        G.add_edge(v1, v2, weight=weight)

In [133]:
G.summary()

'IGRAPH UNW- 412 41261 -- \n+ attr: name (v), weight (e)'

In [146]:
visual_style = {}
visual_style["layout"] = G.layout("kk")

visual_style["vertex_size"] = 20
visual_style["vertex_label"] = G.vs["name"]
visual_style["label_size"] = .5

visual_style["edge_width"] = [1 for e in G.es]


visual_style["bbox"] = (600, 600)
visual_style["margin"] = 20


# ig.plot(G, **visual_style)